# Groq API Tasks: Conversation Management & Information Extraction
**Submitted by: Daksh Agarwal**
***
This notebook demonstrates two core tasks using the Groq API with its OpenAI-compatible SDK, without relying on external frameworks like LangChain.

* **Task 1:** Managing and periodically summarizing a conversation history to maintain context over long interactions.
* **Task 2:** Extracting structured information (e.g., name, email, location) from unstructured chat messages using a JSON schema and the tool-calling feature.

### Instructions for Evaluation
To run this notebook, you must configure your own Groq API key.

1.  In the Colab menu, click the **key icon (Secrets)** on the left sidebar.
2.  Create a new secret with the name `GROQ_API_KEY`.
3.  Paste your personal Groq API key into the **Value** field.
4.  Ensure the **"Notebook access"** toggle is enabled.

In [ ]:
# %%capture
# @title 1. Setup, Configuration, and Dependencies
# Install the necessary Python library for Groq
!pip install groq

import os
from groq import Groq
from google.colab import userdata
import json

print("✅ Setup Complete: Libraries installed and imported.")

# --- API Key Configuration ---
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    os.environ["GROQ_API_KEY"] = groq_api_key
    print("✅ Groq API Key loaded successfully.")
except Exception as e:
    print("🚨 Error loading Groq API Key. Please set it in Colab's Secrets Manager.")
    groq_api_key = None

# --- Centralized Model Configuration ---
# 💡 If you get a 'model decommissioned' error, find a new model name from
# https://console.groq.com/docs/models and update the variable below.
ACTIVE_MODEL = "llama-3.1-8b-instant"

# --- Initialize Groq Client ---
if groq_api_key:
    try:
        client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
        print(f"✅ Groq client initialized. Using model: {ACTIVE_MODEL}")
    except Exception as e:
        client = None
        print(f"🚨 Failed to initialize Groq client: {e}")
else:
    client = None
    print("🚨 Groq client not initialized because API key is missing.")


# %%
# @title 2. Helper Functions (for both tasks)

def format_history_for_display(history):
    """Helper to pretty-print the conversation history."""
    formatted_string = "--- Conversation History ---\n"
    if not history: return formatted_string + "  [Empty]\n"
    for msg in history: formatted_string += f"  [{msg['role'].capitalize()}]: {msg['content']}\n"
    formatted_string += "--------------------------"
    return formatted_string

# --- Task 1 Helper Functions ---

def get_assistant_response(messages):
    """Gets a conversational response from the Groq API."""
    if not client: return "Groq client not initialized."
    try:
        chat_completion = client.chat.completions.create(
            messages=messages,
            model=ACTIVE_MODEL, # Uses the central model variable
            temperature=0.7,
            max_tokens=100,
        )
        return chat_completion.choices[0].message.content
    except Exception as e: return f"Error communicating with Groq API: {e}"

def truncate_by_turns(history, n_turns):
    """Limits the history to the last n conversation turns."""
    return history[-(n_turns * 2):]

def truncate_by_length(history, max_chars):
    """Limits history by character count, starting from the most recent message."""
    truncated_history = []
    current_chars = 0
    for message in reversed(history):
        message_chars = len(message.get('content', ''))
        if current_chars + message_chars <= max_chars:
            truncated_history.insert(0, message)
            current_chars += message_chars
        else: break
    return truncated_history

def summarize_conversation(history):
    """Uses the Groq API to summarize the conversation history."""
    if not client: return "Groq client not initialized."
    conversation_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in history])
    summarization_prompt = [
        {"role": "system", "content": "You are an expert summarizer. Create a concise, neutral summary of the following chat. Capture the main topics, questions, and conclusions."},
        {"role": "user", "content": conversation_text}
    ]
    try:
        chat_completion = client.chat.completions.create(
            messages=summarization_prompt,
            model=ACTIVE_MODEL, # Uses the central model variable
        )
        return chat_completion.choices[0].message.content
    except Exception as e: return f"Error during summarization: {e}"

# --- Task 2 Helper Functions ---

USER_DETAILS_TOOL = {
    "type": "function",
    "function": {
        "name": "extract_user_info",
        "description": "Extracts user details from the text. Only include fields for which a value is explicitly provided. Do not include fields with null values.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "The full name of the user."},
                "email": {"type": "string", "description": "The user's email address."},
                "phone": {"type": "string", "description": "The user's phone number."},
                "location": {"type": "string", "description": "The user's city, state, or country."},
                "age": {"type": "integer", "description": "The age of the user."},
            },
            "required": [],
        },
    },
}

def extract_info_from_chat(chat_message):
    """Uses the Groq API with tool calling to extract structured data."""
    if not client: return None
    try:
        response = client.chat.completions.create(
            model=ACTIVE_MODEL, # Uses the central model variable
            messages=[
                {"role": "system", "content": "Your ONLY task is to use the provided `extract_user_info` tool to capture relevant information. Do not infer other tool names. Focus only on extracting details."},
                {"role": "user", "content": chat_message}
            ],
            tools=[USER_DETAILS_TOOL],
            tool_choice="auto",
        )
        tool_call = response.choices[0].message.tool_calls
        if tool_call: return json.loads(tool_call[0].function.arguments)
        else: return None
    except Exception as e:
        print(f"An error occurred during API call: {e}")
        return None

def validate_extracted_data(data, schema):
    """Performs a simple, manual validation of the extracted data."""
    if not isinstance(data, dict): return False, "Validation Failed: Not a dictionary."
    allowed_properties = schema['function']['parameters']['properties']
    for key, value in data.items():
        if key not in allowed_properties: return False, f"Validation Failed: Unexpected field '{key}'."
        expected_type_str = allowed_properties[key]['type']
        expected_type = {'string': str, 'integer': int}.get(expected_type_str)
        if not isinstance(value, expected_type): return False, f"Validation Failed: Field '{key}' has wrong type."
    return True, "✅ Validation Passed: Extracted data conforms to the schema."


# %%
# @title 3. Task 1: Conversation History and Summarization

class ConversationManager:
    """Manages the conversation history, including periodic summarization."""
    def __init__(self, k_runs_for_summarization=3):
        self.history = []
        self.run_counter = 0
        self.k = k_runs_for_summarization
        print(f"ConversationManager initialized. Summarization will occur after every {self.k} turns.")

    def add_turn(self, user_input):
        print(f"\n--- Turn {self.run_counter + 1} ---")
        self.history.append({"role": "user", "content": user_input})
        assistant_response = get_assistant_response(self.history)
        self.history.append({"role": "assistant", "content": assistant_response})
        print(f"[User]: {user_input}\n[Assistant]: {assistant_response}")
        self.run_counter += 1
        if self.run_counter % self.k == 0:
            print(f"\n>>> Condition met: Run {self.run_counter}. Triggering summarization...")
            self.summarize_and_replace()

    def summarize_and_replace(self):
        print(">>> Current history before summarization:", format_history_for_display(self.history))
        summary = summarize_conversation(self.history)
        print("\n>>> Generated Summary:", summary)
        self.history = [{"role": "system", "content": f"Summary of previous conversation: {summary}"}]
        print("\n>>> History has been replaced with the summary.")

# --- DEMONSTRATION FOR TASK 1 ---
print(" TASK 1 DEMONSTRATION ".center(70, "="))
conversation_flow = [
    "I need to learn about large language models. Where should I start?",
    "What's the difference between fine-tuning and retrieval-augmented generation?",
    "That makes sense. So RAG is better for knowledge-intensive tasks without retraining?",
    "Okay, let's switch topics. I want to build a simple Python web app.",
    "Which framework is easier for a beginner, Flask or Django?",
    "Thanks. I'll start with Flask. Can you give me a 'Hello, World!' example?",
]
manager = ConversationManager(k_runs_for_summarization=3)
if client:
    for user_prompt in conversation_flow:
        manager.add_turn(user_prompt)
        print("\nCurrent History State:", format_history_for_display(manager.history))
else:
    print("\n🚨 Halting Task 1 demonstration because Groq client is not initialized.")

# %%
# @title 4. Task 2: JSON Schema Classification & Information Extraction

# --- DEMONSTRATION FOR TASK 2 ---
print("\n" + " TASK 2 DEMONSTRATION ".center(70, "="))
sample_chats = [
    "Hi, I'm John Doe and I'm 29. My email is john.doe@example.com and you can reach me at (123) 456-7890. I'm currently in New York.",
    "Please sign up Jane Smith for the newsletter at jane.s@email.net.",
    "Hello, I'd like to know more about your services. What do you offer?",
    "My name is Priya Sharma, I live in Mumbai. I am 35 years old. My phone is +91 98765 43210."
]

if client:
    for i, chat in enumerate(sample_chats):
        print(f"\n--- Processing Sample {i+1} ---")
        print(f"Input Chat: \"{chat}\"")
        extracted_data = extract_info_from_chat(chat)
        if extracted_data:
            print("\n🔍 Extracted Information:", json.dumps(extracted_data, indent=2))
            is_valid, message = validate_extracted_data(extracted_data, USER_DETAILS_TOOL)
            print(f"\nValidation Result: {message}")
        else:
            print("\n✅ Result: No structured information was extracted, as expected for this input.")
else:
    print("\n🚨 Halting Task 2 demonstration because Groq client is not initialized.")

print("\n" + "="*70)
print("✅ All Demonstrations Finished.")

✅ Setup Complete: Libraries installed and imported.
✅ Groq API Key loaded successfully.
✅ Groq client initialized. Using model: llama-3.1-8b-instant
======================== TASK 1 DEMONSTRATION ========================
ConversationManager initialized. Summarization will occur after every 3 turns.

--- Turn 1 ---
[User]: I need to learn about large language models. Where should I start?
[Assistant]: Large language models (LLMs) are a rapidly evolving field in natural language processing (NLP), and there's a lot to learn. Here's a step-by-step guide to help you get started:

**1. Understand the basics of NLP**:
Begin by learning about the fundamentals of NLP, including:
	* Tokenization
	* Part-of-speech tagging
	* Named entity recognition
	* Sentiment analysis
	* Text classification

You can find many online resources

Current History State: --- Conversation History ---
  [User]: I need to learn about large language models. Where should I start?
  [Assistant]: Large language models (LLM